In [11]:
# !pip3 install chardet -i https://mirrors.aliyun.com/pypi/simple/

In [135]:
import glob
import os
from tqdm import tqdm
import numpy as np
import shutil
import random

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [221]:
sr = 16000
result_base_dir = "./audio/20240815_spaccent/"
speakers = ["BWC", "LXC", "NCC", "TXHC"]

experiments = {
    "mushra_naturalness": [
        "source",
        "resynth",
        "pseudo",
        "ours",
        "ours-scale",
        "ours-control-scale",
        "baseline",
    ],
    "mushra_accentedness": [
        "source",
        "resynth",
        "pseudo",
        "ours",
        "ours-scale",
        "ours-control-scale",
        "baseline",
    ],
    "bws_similarity": [
        "source", # reference
        "ours",
        "ours-scale",
        "ours-control-scale",
        "baseline",
    ],
    "bws_naturalness": [ # duration control
        "ours-scale",
        "ours-control-scale",
    ],
}


import chardet
def openfile(txtfile):
    with open(txtfile, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(txtfile, "r", encoding=encoding) as f:
        content = f.readlines()
    return content
files = glob.glob("../../seq2seq-vc/datasetgeneration/LLM_responses/08-A_multi-lingual_text/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = text

In [222]:
nsamples_per_test = {
    "mushra_naturalness": 2,
    "mushra_accentedness": 2,
    "bws_similarity": 3, 
    "bws_naturalness": 3, 
}
testnames = {
    "mushra_naturalness": "Speech Naturalness Evaluation (MUSHRA)",
    "mushra_accentedness": "Speech Accentedness Evaluation (MUSHRA)",
    "bws_similarity": "Speaker Similarity Evaluation (BWS)", 
    "bws_naturalness": "Speech Naturalness Evaluation (BWS)", 
}
testids = {
    "mushra_naturalness": "MUSHRANAT",
    "mushra_accentedness": "MUSHRAACT",
    "bws_similarity": "BWSSIM", 
    "bws_naturalness": "BWSNAT", 
}

# MUSHRA

In [227]:
save = True
for exid in ["mushra_naturalness", "mushra_accentedness"]:
# for exid in ["mushra_naturalness"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    pngfile = "img/scale_natural.png" if testmode=="naturalness" else "img/scale_accent.png"
    # pngfile = "img/scale_abs.png" if testmode=="naturalness" else "img/scale_accent.png"
    def init():
        whole_text = f"""var TestConfig = {{
      "TestName": "{testname}",
      "RateScalePng": "{pngfile}",
      "RateScaleBgPng": "img/scale_abs_background.png",
      "RateMinValue": 0,
      "RateMaxValue": 100,
      "RateDefaultValue":0,
      "ShowFileIDs": true,
      "ShowResults": false,
      "LoopByDefault": false,
      "EnableABLoop": true,
      "EnableOnlineSubmission": false,
      "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
      "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
      "RandomizeTestOrder": true,
      "MaxTestsPerRun": {nsamples_per_test[exid]},
      "RequireMaxRating": false,
      "AudioRoot": "",
      "Testsets":
    ["""
        return whole_text

    whole_text = init()
    dirnames = experiments[exid]
    dirname = dirnames[0]
    for spk in speakers:
        filenames = [os.path.basename(a)[:-4] for a in glob.glob(result_base_dir+f"{dirname}_{spk}/*.wav")]
        filenames.sort()
        for b, basename in enumerate(filenames):
            _, name = basename.split("_")
            text = groundtruth_dir[basename]
            starter = f"""
      {{
        "Name": "{basename}",
        "TestID": "{testid}---{basename}---{spk}",
        "Text": "{text}",
        "Files": {{\n"""
            modelpaths = """"""
            for dirname in dirnames:
                modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
                added = f'      "{dirname}": "{modelpath}",\n'
                modelpaths += added
            ##### One more sample #####
            np.random.seed(b)
            dirname = dirnames[np.random.randint(len(dirnames))]
            modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
            added = f'      "{dirname}---2": "{modelpath}",\n'
            modelpaths += added
            ##########################
            modelpaths += f'    }}\n'
            modelpaths += f'  }},'
            whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

var TestConfig = {
      "TestName": "Speech Naturalness Evaluation (MUSHRA)",
      "RateScalePng": "img/scale_natural.png",
      "RateScaleBgPng": "img/scale_abs_background.png",
      "RateMinValue": 0,
      "RateMaxValue": 100,
      "RateDefaultValue":0,
      "ShowFileIDs": true,
      "ShowResults": false,
      "LoopByDefault": false,
      "EnableABLoop": true,
      "EnableOnlineSubmission": false,
      "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
      "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
      "RandomizeTestOrder": true,
      "MaxTestsPerRun": 2,
      "RequireMaxRating": false,
      "AudioRoot": "",
      "Testsets":
    [
      {
        "Name": "arctic_a0017",
        "TestID": "MUSHRANAT---arctic_a0017---BWC",
        "Text": "From that moment his friendship for Belize turns to hatred and jealousy.",
        "Files": {
      "source": "./audio/20240815_spaccent/source_BWC/arctic_a0017.wav",
      "resynth": "./audio/20240815_spaccent/resynt

# BWS Test

In [228]:
instructions = {
    "bws_similarity":{
        "Low": "the 'LEAST' similar'",
        "High": "the 'MOST' similar'",
    },
    "bws_naturalness":{
        "Low": "the 'LEAST' natural'",
        "High": "the 'MOST' natural'",
    },
}

In [230]:
BWS (3 options): 48 samples for 20 mins

SyntaxError: invalid syntax (223251532.py, line 1)

In [229]:
save = True
bws_labels = ["V", "W", "X", "Y", "Z"][::-1]
evaluation_filenames = [os.path.basename(a)[:-4] for a in glob.glob("./audio/20240815_spaccent/source_TXHC/*")]
evaluation_filenames.sort()
for exid in ["bws_similarity", "bws_naturalness"]:
    with_reference = exid in ["bws_similarity"]
# for exid in ["bws_similarity"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    instruction = instructions[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    np.random.seed(0)
    lists = list(np.random.choice(evaluation_filenames, size=nsamples_per_test[exid]))
    lists.sort()
    for basename in lists:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "LowText": "{instruction['Low']}",
    "HighText": "{instruction['High']}",
    "Length": {len(modelnames)-int(with_reference)},
    "Files": {{\n"""
        modelpaths = """"""
        for d, dirname in enumerate(modelnames):
            modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
            if with_reference and d==0:
                label = "Reference"
            else:
                label = bws_labels[d-int(with_reference)]
            added = f'      "{label}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

var TestConfig = {
  "TestName": "Speaker Similarity Evaluation (BWS)",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "arctic_b0344",
    "TestID": "BWSSIM---arctic_b0344---NCC",
    "Text": "Lots of men take women buggy riding.",
    "LowText": "the 'LEAST' similar'",
    "HighText": "the 'MOST' similar'",
    "Length": 4,
    "Files": {
      "Reference": "./audio/20240815_spaccent/source_NCC/arctic_b0344.wav",
      "Z": "./audio/20240815_spaccent/ours_NCC/arctic_b0344.wav",
      "Y": "./audio/20240815_spaccent/ours-scale_NCC/arctic_b0344.wav",
      "X": "./audio/20240815_spaccent/ours-control-scale_NCC/arctic_b0344.wav",
      "W": "./audio/20240815_spaccent/baseline_NCC/arctic_b0344.wav",
    }
  },
  {
    "Name": "arctic_b0031",
   

----
----
----
----
----
----
----
----
----
----
----

In [165]:
num = 100000
start = 0
save = True
with_reference = False
evaluation_filenames = ["arctic_a0024", "arctic_a0029"]
bws_labels = ["W", "X", "Y", "Z"][::-1]
# for exid in ["bws_similarity", "bws_naturalness"]:
for exid in ["bws_similarity"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    for basename in evaluation_filenames:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "Length": {len(modelnames)-int(with_reference)},\n"""
        if with_reference:
            dirname = modelnames[0]
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            starter += f"""
    "Reference": "{modelpath}",
    "Files": {{\n"""[1:]
        else:
            starter += f"""
    "Files": {{\n"""[1:]
        modelpaths = """"""
        for d, dirname in enumerate(modelnames[int(with_reference):]):
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            added = f'      "{bws_labels[d]}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

IndexError: list index out of range

In [144]:
num = 100000
start = 0
save = True
evaluation_filenames = ["arctic_a0024", "arctic_a0029"]
bws_labels = ["W", "X", "Y", "Z"][::-1]
# for exid in ["bws_similarity", "bws_naturalness"]:
for exid in ["bws_similarity"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    for basename in evaluation_filenames:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "Length": {len(modelnames)},
    "Files": {{\n"""
        modelpaths = """"""
        for d, dirname in enumerate(modelnames):
            dirname = f'{"___".join(dirname.split("/"))}'
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            added = f'      "{bws_labels[d]}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

var TestConfig = {
  "TestName": "Speaker Similarity Evaluation (BWS)",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "arctic_a0029",
    "TestID": "BWSSIM---arctic_a0029---SLT",
    "Text": "Their forces were already moving into the north country.",
    "Length": 3,
    "Files": {
      "Z": "./audio/20240815_spaccent/Model1___SLT/arctic_a0029.wav",
      "Y": "./audio/20240815_spaccent/Model2___SLT/arctic_a0029.wav",
      "X": "./audio/20240815_spaccent/Model3___SLT/arctic_a0029.wav",
    }
  },
  {
    "Name": "arctic_a0024",
    "TestID": "BWSSIM---arctic_a0024---ASI",
    "Text": "It was my reports from the north which chiefly induced people to buy.",
    "Length": 3,
    "Files": {
      "Z": "./audio/20240815_spaccent/Model1___ASI/ar

In [130]:
import random

emos = ["Angry", "Happy", "Sad", "Surprise"]

def init():
    whole_text = """var TestConfig = {
  "TestName": "Utterance-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[\n"""
    return whole_text

evaluation_filenames = [
    # "0013_000731",
    "0014_000732",
    "0015_000726",
    # "0018_000037",
]

In [131]:
num = 10000000
start = 0
whole_text = init()
result_dir = result_base_dir + f"utterance_remain/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
filenames = glob.glob(f"{result_dir}{modelnames[0]}/*-Angry-0.0-0.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    basename, _, _ = basename.split("-")
    if not (basename in evaluation_filenames):
        continue
    spk, name = basename.split("_")
    for emotion in emos:
        for mn in modelnames:
            if "msemotts" in mn:
                continue
            starter = "  {\n"
            starter += f'    "Name": "{basename}-{emotion}",\n'
            starter += f'    "TestID": "{mn}---{basename}---{emotion}",\n'
            starter += f'    "Text": "Intended Emotion: <b>{emotion}</b>",\n'
            starter += f'    "Files": {{\n'
            modelpaths = """"""
            for itt, intensity in enumerate(intensities):
                modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
                added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
                modelpaths += added
            modelpaths += f'    }}\n'
            modelpaths += f'  }},\n'
            whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])

KeyError: 'main'